# Evaluation of RAG architectures

In [ ]:
from tqdm import tqdm
from pathlib import Path
from llms.clients.gpt import GPTClient
from llms.settings import settings
from llms.rag.faiss import DistanceMetric
from llms.evaluation.code import evaluate_code_generation, ConfigGrid, RAG, RAGRetriever
from tests.pandas import TEST_CASES

In [ ]:
def get_texts(folder_paths: list[str]) -> list[str]:
    folder_paths = [Path(folder_path) for folder_path in folder_paths]
    files = []
    for folder_path in tqdm(folder_paths, desc="Searching for files"):
        for child in folder_path.iterdir():
            if child.is_file():
                files.append(child)
    texts = []
    for file in tqdm(files, desc="Reading files"):
        with open(file, "r", encoding="utf-8") as f:
            content = f.read()
            texts.append(content)
    return texts

In [ ]:
folder_paths = ["files/pandas/textfiles/textfiles1", "files/pandas/textfiles/textfiles2", "files/pandas/textfiles/textfiles3"]
texts = get_texts(folder_paths)

In [ ]:
gpt_4_client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id='gpt-4-32k',
    max_response_tokens=1000,
    temperature=0.0,
)

In [ ]:
config_grid = ConfigGrid(
    llms=[gpt_4_client],
    rag=RAG(
        retrievers=[
            RAGRetriever.NONE,
            RAGRetriever.RAG,
            RAGRetriever.RAG_AS_TOOL,
            RAGRetriever.CoALA,
            RAGRetriever.CoALA_AS_TOOL,
        ],
        distance_metrics=[DistanceMetric.EUCLIDEAN_DISTANCE, DistanceMetric.MAX_INNER_PRODUCT],
        num_search_results=[3],
        similarity_search_score_thresholds=[0.0],
        text_chunk_sizes=[512],
        use_weighted_average_of_text_chunks=[True],
        texts=texts,
    )
)

In [ ]:
results = evaluate_code_generation(config_grid=config_grid, test_cases=TEST_CASES, test_name="pandas_test")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path = "./results"
df = pd.read_csv(f"{path}/pandas_results.csv")

In [ ]:
df = df.reset_index()
df

In [ ]:
df.plot(kind='bar', x='index', y='accuracy', legend=False)
plt.title('Accuracy for solving test cases')
plt.xlabel('Id')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
df.plot(kind='bar', x='index', y='total_cost', legend=False)
plt.title('Total cost for solving test cases')
plt.xlabel('Id')
plt.ylabel('Total Cost in $')
plt.show()

In [ ]:
df.plot(kind='bar', x='index', y='total_time', legend=False)
plt.title('Total time for solving test cases')
plt.xlabel('Id')
plt.ylabel('Total Time in seconds')
plt.show()

In [ ]:
details_path = df.iloc[0].details_csv_filepath
details_df = pd.read_csv(details_path)
details_df